To run this Fenic demo, click **Runtime** > **Run all**.

<div class="align-center">
<a href="https://github.com/typedef-ai/fenic"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/typedef-fenic-logo-github-yellow.png" height="50"></a>
<a href="https://discord.gg/GdqF3J7huR"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/join-the-discord.png" height="50"></a>
<a href="https://docs.fenic.ai/latest/"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/documentation.png" height="50"></a>

Questions? Join the Discord and ask away! For feature requests or to leave a star, visit our [GitHub](https://github.com/typedef-ai/fenic).

</div>

In [ ]:
!pip uninstall -y sklearn-compat ibis-framework imbalanced-learn google-genai
!pip install polars==1.30.0 numpy-financial
# === GOOGLE GEMINI ===
#!pip install fenic[google]
# === ANTHROPIC CLAUDE ===
#!pip install fenic[anthropic]
# === OPENAI (Default) ===
!pip install fenic

In [ ]:
import os 
import getpass

# 🔌 MULTI-PROVIDER SETUP - Choose your preferred LLM provider
# Uncomment ONE of the provider sections below:

# === OPENAI (Default) ===
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

# === GOOGLE GEMINI ===
# os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google API Key:")

# === ANTHROPIC CLAUDE ===
# os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Anthropic API Key:")

# 🛠️ Custom UDFs with External Libraries

**Hook:** *"Extend Fenic with any Python library - AI meets specialized domain expertise"*

Fenic expressions can handle most data operations, but sometimes you need specialized libraries for domain-specific calculations. Custom UDFs let you integrate any Python library - financial modeling with numpy-financial, text analysis with spaCy, image processing with PIL.

**What you'll see in this 2-minute demo:**
- 🏦 **Financial loan data** - Applications with rates, terms, and narrative notes
- 📊 **numpy-financial UDF** - Precise loan calculations Fenic can't do natively
- 🧠 **AI + External library** - Semantic analysis combined with specialized calculations
- 🔧 **Extensible framework** - Any Python library can be integrated

Perfect for financial services, scientific computing, and domain-specific workflows.

In [ ]:
import fenic as fc
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Literal
import numpy_financial as npf

# 🛠️ Configure session for custom UDF development
session = fc.Session.get_or_create(fc.SessionConfig(
    app_name="custom_udfs_demo",
    semantic=fc.SemanticConfig(
        language_models={
            "business_analyst": fc.OpenAILanguageModel(model_name="gpt-4o", rpm=100, tpm=50_000),
            # "business_analyst": fc.GoogleDeveloperLanguageModel(model_name="gemini-2.5-flash-lite", rpm=1000, tpm=1_000_000),
            # "business_analyst": fc.AnthropicLanguageModel(model_name="claude-3-5-sonnet-20241022", rpm=500, tpm=200_000)
        }
    )
))

print("✅ Custom UDF development session configured")
print("   • External Library: numpy-financial for advanced loan calculations")

## 🏦 Step 1: Financial Dataset with Loan Parameters

Loan applications with structured financial data and unstructured notes:

In [ ]:
# 🏦 Loan applications with financial parameters for numpy-financial calculations
loan_applications = session.create_dataframe([
    {
        "application_id": "APP001",
        "applicant_name": "Sarah Johnson",
        "annual_income": 85000,
        "loan_amount": 250000,
        "interest_rate": 6.5,
        "loan_years": 30,
        "application_notes": "Stable software engineer at tech startup. Recently promoted to senior role. Planning first home purchase with spouse. 20% down payment saved."
    },
    {
        "application_id": "APP002",
        "applicant_name": "Mike Rodriguez", 
        "annual_income": 120000,
        "loan_amount": 400000,
        "interest_rate": 7.25,
        "loan_years": 30,
        "application_notes": "Marketing director with strong track record. Recently divorced, paying alimony. Stock options vest next year. Previous foreclosure 7 years ago."
    },
    {
        "application_id": "APP003",
        "applicant_name": "Jennifer Chen",
        "annual_income": 95000,
        "loan_amount": 180000,
        "interest_rate": 6.0,
        "loan_years": 15,
        "application_notes": "Government employee with excellent job security. Military veteran with VA benefits. Refinancing to lower rates. Excellent payment history."
    },
    {
        "application_id": "APP004",
        "applicant_name": "David Park",
        "annual_income": 75000,
        "loan_amount": 300000,
        "interest_rate": 8.0,
        "loan_years": 30,
        "application_notes": "Self-employed contractor with variable income. Business grown significantly in past 2 years. Filed Chapter 13 bankruptcy 5 years ago."
    }
])

print("🏦 Loan Application Dataset:")
print("   • Financial parameters for numpy-financial calculations")
print("   • Unstructured notes for AI risk analysis")
loan_applications.select("application_id", "applicant_name", "loan_amount", "interest_rate", "loan_years").show()

## 🧠 Step 2: AI Risk Analysis Schema

Define structured insights to extract from loan application notes:

In [ ]:
# 🧠 AI risk assessment schema
class RiskAssessment(BaseModel):
    employment_stability: Literal["excellent", "good", "fair", "poor"] = Field(description="Job security assessment")
    financial_complexity: Literal["simple", "moderate", "complex", "very_complex"] = Field(description="Situation complexity level")
    risk_factors: List[str] = Field(description="Potential risk indicators from notes")
    stability_factors: List[str] = Field(description="Positive stability indicators")
    narrative_risk_score: float = Field(description="Risk score 0.0-10.0 (lower = better)")

print("🧠 AI Risk Assessment Schema:")
print("   • employment_stability: Job security evaluation")
print("   • financial_complexity: Situation assessment")
print("   • risk_factors: AI-identified concerns")
print("   • stability_factors: AI-identified strengths")
print("   • narrative_risk_score: Quantified narrative risk")

## 🔧 Step 3: Custom UDF with numpy-financial

Create a UDF using numpy-financial for calculations Fenic can't do natively:

In [ ]:
# 🔧 Custom UDF using numpy-financial for advanced loan calculations
@fc.udf(return_type=fc.StructType([
    fc.StructField("monthly_payment", fc.DoubleType),
    fc.StructField("total_interest", fc.DoubleType),
    fc.StructField("payment_to_income_ratio", fc.DoubleType),
    fc.StructField("affordability_rating", fc.StringType),
    fc.StructField("rate_sensitivity", fc.DoubleType),
    fc.StructField("first_year_principal_pct", fc.DoubleType)
]))
def calculate_loan_metrics(
    loan_amount: float,
    annual_rate: float, 
    loan_years: int,
    annual_income: float
) -> Dict[str, Any]:
    """
    Advanced loan calculations using numpy-financial that Fenic expressions cannot do
    """
    # Convert to monthly terms
    monthly_rate = annual_rate / 100.0 / 12.0
    num_payments = loan_years * 12
    
    # Use numpy-financial for precise payment calculation
    monthly_payment = float(-npf.pmt(monthly_rate, num_payments, loan_amount))
    
    # Calculate total interest over loan life
    total_paid = monthly_payment * num_payments
    total_interest = float(total_paid - loan_amount)
    
    # Payment-to-income ratio
    monthly_income = annual_income / 12.0
    payment_ratio = float((monthly_payment / monthly_income) * 100)
    
    # Affordability assessment
    if payment_ratio <= 28:
        affordability = "excellent"
    elif payment_ratio <= 36:
        affordability = "good"
    elif payment_ratio <= 43:
        affordability = "marginal"
    else:
        affordability = "high_risk"
    
    # Interest rate sensitivity (payment change with 1% rate increase)
    higher_rate = (annual_rate + 1.0) / 100.0 / 12.0
    higher_payment = float(-npf.pmt(higher_rate, num_payments, loan_amount))
    sensitivity = float(((higher_payment - monthly_payment) / monthly_payment) * 100)
    
    # First year principal percentage using numpy-financial
    first_year_interest = 0
    for month in range(1, 13):
        interest_payment = float(-npf.ipmt(monthly_rate, month, num_payments, loan_amount))
        first_year_interest += interest_payment
    
    first_year_payments = monthly_payment * 12
    first_year_principal = first_year_payments - first_year_interest
    principal_percentage = float((first_year_principal / first_year_payments) * 100)
    
    return {
        "monthly_payment": round(monthly_payment, 2),
        "total_interest": round(total_interest, 2),
        "payment_to_income_ratio": round(payment_ratio, 2),
        "affordability_rating": affordability,
        "rate_sensitivity": round(sensitivity, 2),
        "first_year_principal_pct": round(principal_percentage, 2)
    }

print("🔧 numpy-financial UDF registered successfully!")
print("   • Uses numpy-financial PMT, IPMT functions")
print("   • Provides calculations Fenic expressions cannot do")
print("   • Demonstrates external library integration")

## 🚀 Step 4: Combined AI + External Library Pipeline

Execute semantic analysis alongside numpy-financial calculations:

In [ ]:
# 🚀 Execute combined AI + numpy-financial pipeline
print("🚀 Starting combined semantic + external library pipeline...")

# Step 1: AI narrative analysis
with_ai_analysis = loan_applications.select(
    "*",
    fc.semantic.extract(
        "application_notes",
        RiskAssessment,
        model_alias="business_analyst"
    ).alias("ai_risk")
).cache()

print("✅ Step 1: AI narrative analysis completed")

# Step 2: numpy-financial calculations
comprehensive_analysis = with_ai_analysis.select(
    "*",
    calculate_loan_metrics(
        fc.col("loan_amount"),
        fc.col("interest_rate"),
        fc.col("loan_years"),
        fc.col("annual_income")
    ).alias("loan_metrics")
).cache()

print("✅ Step 2: numpy-financial calculations completed")

# Step 3: Extract results for analysis
final_results = comprehensive_analysis.select(
    "application_id",
    "applicant_name",
    "loan_amount",
    "interest_rate",
    comprehensive_analysis.ai_risk.employment_stability.alias("employment_stability"),
    comprehensive_analysis.ai_risk.narrative_risk_score.alias("narrative_risk"),
    comprehensive_analysis.loan_metrics.monthly_payment.alias("monthly_payment"),
    comprehensive_analysis.loan_metrics.affordability_rating.alias("affordability"),
    comprehensive_analysis.loan_metrics.rate_sensitivity.alias("rate_sensitivity"),
    comprehensive_analysis.loan_metrics.first_year_principal_pct.alias("first_year_principal_pct")
)

print("\n🏦 COMBINED AI + numpy-financial RESULTS:")
final_results.show()

print("✅ Combined semantic + external library pipeline completed!")

## 📊 Step 5: Business Intelligence Analysis

Analyze the combined AI and numpy-financial insights:

In [ ]:
# 📊 Business intelligence with combined insights
print("📊 BUSINESS INTELLIGENCE DASHBOARD")
print("=" * 50)

# Affordability distribution (numpy-financial powered)
affordability_analysis = final_results.group_by("affordability").agg(
    fc.count("*").alias("count"),
    fc.avg("monthly_payment").alias("avg_payment"),
    fc.avg("rate_sensitivity").alias("avg_sensitivity")
).order_by(fc.desc("count"))

print("\n💼 AFFORDABILITY ANALYSIS (numpy-financial):")
affordability_analysis.show()

# Risk insights
total_applications = final_results.count()
high_risk_payments = final_results.filter(fc.col("affordability") == "high_risk").count()
high_sensitivity = final_results.filter(fc.col("rate_sensitivity") > 7.0).count()

print("\n🎯 KEY INSIGHTS:")
print(f"   • Total applications: {total_applications}")
print(f"   • High-risk payment ratios: {high_risk_payments} ({high_risk_payments/total_applications*100:.1f}%)")
print(f"   • High rate sensitivity (>7%): {high_sensitivity} ({high_sensitivity/total_applications*100:.1f}%)")

# Show detailed risk factors
risk_details = comprehensive_analysis.select(
    "application_id",
    "applicant_name",
    comprehensive_analysis.ai_risk.risk_factors.alias("ai_risk_factors"),
    comprehensive_analysis.ai_risk.stability_factors.alias("ai_stability_factors")
).limit(2)

print("\n🔍 AI RISK FACTOR ANALYSIS (Sample):")
risk_details.show()

print("\n🛠️ EXTERNAL LIBRARY UDF BREAKTHROUGH:")
print("   ✅ numpy-financial provides precise calculations Fenic can't do")
print("   ✅ PMT, IPMT functions for accurate loan modeling")
print("   ✅ Combined with AI semantic analysis")
print("   ✅ Extensible to any Python library")

print("\n💡 ADDITIONAL UDF POSSIBILITIES:")
print("   • scipy: Advanced statistical modeling")
print("   • scikit-learn: Custom ML models")
print("   • requests: Real-time API integrations")
print("   • Any domain-specific Python library")

In [ ]:
session.stop()